In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.green_security_game import GreenSecurityGame

In [2]:
import numpy as np

def compute_utility(utilities, strategy, second_player_utilities = None):
    eus = [[0 for _ in range(len(utilities))], [0 for _ in range(len(utilities[0]))]]
    for i in range(len(utilities)):
        for j in range(len(utilities[0])):
            eus[0][i] += utilities[i][j] * strategy[1][j]
            if second_player_utilities == None:
                eus[1][j] -= utilities[i][j] * strategy[0][i]
            else:
                eus[1][j] += second_player_utilities[i][j] * strategy[0][i]
    return eus


def get_averaging_denom(player, iterations_made, averaging):
    if averaging == 0:
        return 1.0 / iterations_made[player]
    elif averaging == 1:
        return 2.0 / ( iterations_made[player] * (iterations_made[player] + 1) )
    elif averaging == 2:
        return 6.0 / ( iterations_made[player] * (iterations_made[player] + 1) * (2 * iterations_made[player] + 1) )
    else:
        return None

def regret_matching(utilities, second_player_utilities = None, iterations=10000, averaging = 2, alternations = True, plus = True, predictive = True, verbose=False, precision = 1e-2):
    n_players = 2
    m_actions = [len(utilities), len(utilities[0])]

    regrets   = [[0] * m_actions[i] for i in range(n_players)]
    strategy_sum = [[0] * m_actions[i] for i in range(n_players)]
    strategy  = [[1 / m_actions[i]] * m_actions[i] for i in range(n_players)]

    players = [0] if alternations else [_ for _ in range(n_players)]

    iterations_made = [0 for _ in range(n_players)]

    gaps = []
    
    for iter_idx in range(iterations):
        if verbose and iter_idx % 100 == 0 and iter_idx > 0:
            average_strategy = [[s * get_averaging_denom(p, iterations_made, averaging) for s in player] for p, player in enumerate(strategy_sum)]
            action_utilities = compute_utility(utilities, average_strategy, second_player_utilities = second_player_utilities)
            
            if second_player_utilities == None:
                dual_gap = max(action_utilities[0]) + max(action_utilities[1])
            else:
                expected_utilities = [sum([strategy[p][a] * action_utilities[p][a] for a in range(m_actions[p])]) for p in range(n_players)]
                dual_gap = sum([max(action_utilities[p]) - expected_utilities[p] for p in range(n_players)])

            print('Iteration', iter_idx, 'gap: ',dual_gap)
            # for i in range(n_players):
            #     print('\t player', i, 'strategy:', average_strategy[i])

            gaps.append(dual_gap)

            if dual_gap < precision: break

        action_utilities = compute_utility(utilities, strategy, second_player_utilities = second_player_utilities)
        expected_utilities = [sum([strategy[p][a] * action_utilities[p][a] for a in range(m_actions[p])]) for p in range(n_players)]
      
        for i in players:
            for a in range(m_actions[i]):
                regrets[i][a] = max(0 if plus else float("-inf"), regrets[i][a] + action_utilities[i][a] - expected_utilities[i])
        
        for i in players:
            positive_regrets = [max(0, (regrets[i][a] + action_utilities[i][a] - expected_utilities[i]) if predictive else regrets[i][a]) for a in range(m_actions[i])]
            normalizing_sum = sum(positive_regrets)
            strategy[i] = [r / normalizing_sum if normalizing_sum > 0 else 1 / m_actions[i] for r in positive_regrets]
        
        for i in players:
            iterations_made[i] += 1
            for a in range(m_actions[i]):
                strategy_sum[i][a] += pow(iterations_made[i], averaging) * strategy[i][a]

        players = [ (iter_idx + 1) % n_players ] if alternations else [_ for _ in range(n_players)]
    
    average_strategy = [[s * get_averaging_denom(p, iterations_made, averaging) for s in player] for p, player in enumerate(strategy_sum)]
    action_utilities = compute_utility(utilities, average_strategy, second_player_utilities = second_player_utilities)
    if verbose: print()
    return average_strategy, action_utilities, gaps




In [ ]:
n = 1000
m = 50
seed = 2
iterations = 100000
plus = True
pred = True
alt = True
av = 2 # 0, 1, or 2 
verbose = True


np.random.seed(seed)
utilities = np.random.randint(-100, 101, size=(n, m))

avg_strategy, exp_utilities, gaps = regret_matching(utilities, iterations = iterations, plus = plus, predictive = pred, alternations = alt, averaging = av, verbose = verbose)
print("Regret matching average strategies:")
for i in range(2):
    print('\t player', i, 'strategy:', avg_strategy[i])
# ... (2 lines left)

In [3]:
def compute_least_exploitable_pure_strategies(payoffs):
    nb_actions = payoffs.shape

    row_max = np.max(payoffs, axis=0)
    row_exploit = row_max - payoffs

    col_min = np.min(payoffs, axis=1)
    col_exploit = payoffs - col_min[:, np.newaxis]

    final_exploitability = row_exploit + col_exploit

    flat_index = np.argmin(final_exploitability)
    coordinates = np.unravel_index(flat_index, final_exploitability.shape)
    coordinates = [int(x) for x in coordinates]

    return (float(np.min(final_exploitability)), coordinates)


In [13]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
lat_min, lon_min = 2.05522, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": False,
    "attacker_animal_value":  2150, 
    "defender_animal_value": 10000, 
    "defender_step_cost": 0, 
    "simple": False,
    "attacker_penalty_factor": 4,
    "defender_penalty_factor": 4,
    "extra_coverage_weight":0.9
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=5, num_rows=5, num_columns=5)
gsg.generate(num_attackers=2, num_defenders=2, home_base_assignments=[(3,3),(1,1)], num_timesteps=6, generate_utility_matrix=True, general_sum=False, **schedule_form_kwargs)

In [8]:
u = gsg.schedule_form_dict["defender_utility_matrix"]

In [14]:
gsg.utility_matrix

array([[-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969],
       [-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969],
       [-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969],
       ...,
       [-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969],
       [-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969],
       [-1.27158774, -1.20612813, -1.29387187, ..., -0.73816156,
        -0.29387187, -0.44428969]])

In [15]:
avg_strategy, exp_utilities, gaps = regret_matching(gsg.utility_matrix, iterations = 10000, plus = True, predictive = True, alternations = True, averaging = 2, verbose = True)

Iteration 100 gap:  0.13142980426333123
Iteration 200 gap:  0.06006427343166898
Iteration 300 gap:  0.02594044200713924
Iteration 400 gap:  0.010974190634993997
Iteration 500 gap:  0.00563242002955644



In [16]:
exp_utilities

[[-0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558454072614108,
  -0.49540571202464834,
  -0.3558095974607448,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558454072614108,
  -0.3558454072614108,
  -0.3558454072614108,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558454072614108,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558454072614108,
  -0.3558454072614108,
  -0.49540571202464834,
  -0.3558095974607448,
  -0.3558095974607448,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558095974607448,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558095974607448,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.3558095974607448,
  -0.3558095974607448,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.49540571202464834,
  -0.4

In [18]:
compute_least_exploitable_pure_strategies(gsg.utility_matrix)

(0.20612813370473537, [14901, 6])